In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import copy
np.set_printoptions(linewidth =np.inf,threshold=np.inf)
import networkx as nx
from collections import defaultdict

class TPMaster:
    def __init__(self):
        self.canevas = {}
        self.G = nx.Graph()
        self.tp = defaultdict(set)
        self.start_end = []

    def _map(self):
        min_x = min(self.canevas, key=lambda k: k[0])[0]
        max_x = max(self.canevas, key=lambda k: k[0])[0]
        min_y = min(self.canevas, key=lambda k: k[1])[1]
        max_y = max(self.canevas, key=lambda k: k[1])[1]

        offset = (abs(min_x), abs(min_y))
        canevas = np.empty((max_x - min_x + 1, max_y - min_y + 1), dtype=str)

        for index, char in self.canevas.items():
            canevas[tuple([sum(x) for x in zip(offset, index)])] = char
        
        for s in self.tp.values():
            for pos in s:
                canevas[pos] = 'X'
                 
        print(canevas)
    
    def parse_map(self, path):
        with open(path, "r") as f:
            i = 0
            for line in f:
                if line != "":
                    for j, char in enumerate(line):
                        if char == "\n":
                            continue
                        self.canevas[(i, j)] = char

                        if char == ".":
                            self.G.add_node((i, j))
                            if self.canevas.get((i-1, j), "#") == ".":
                                self.G.add_edge((i-1, j), (i, j))
                            if self.canevas.get((i, j-1), "#") == ".":
                                self.G.add_edge((i, j-1), (i, j))

                    i += 1
                    
    def add_tp(self):
        for pos, char in self.canevas.items():
            i, j = pos
            if char.isalpha():
                if self.canevas.get((i-1, j), '?').isalpha() and self.canevas.get((i-2, j), '?') == '.':
                    self.tp[(self.canevas[i-1, j], char)].add((i-2, j))
                if self.canevas.get((i+1, j), '?').isalpha() and self.canevas.get((i+2, j), '?') == '.':
                    self.tp[(char, self.canevas[i+1, j])].add((i+2, j))
                if self.canevas.get((i, j-1), '?').isalpha() and self.canevas.get((i, j-2), '?') == '.':
                    self.tp[(self.canevas[i, j-1], char)].add((i, j-2))
                if self.canevas.get((i, j+1), '?').isalpha() and self.canevas.get((i, j+2), '?') == '.':
                    self.tp[(char, self.canevas[i, j+1])].add((i, j+2))
            
        for tps, positions in self.tp.items():
            if len(positions) > 1:
                self.G.add_edge(*list(positions))
            else:
                self.start_end.extend(list(positions))
                
        
    def part_1(self, path):
        self.parse_map(path)
        self.add_tp()
        self._map()
        print(self.tp)
        return nx.shortest_path_length(self.G, *self.start_end)

In [ ]:
tpm = TPMaster()
tpm.part_1("/home/bchatillon/Documents/Advent-of-Code/2019/20_data_test3.txt")

In [ ]:
import numpy as np
import copy
np.set_printoptions(linewidth =np.inf,threshold=np.inf)
import networkx as nx
from collections import defaultdict

class TPMaster2:
    def __init__(self):
        self.canevas = {}
        self.G = nx.Graph()
        self.tp = defaultdict(list)
        self.start_end = []
        self.nb_levels = 40
        self.offset = (0, 0)
        self.len_x = 0
        self.len_y = 0

    def _map(self):
        canevas = np.empty((self.len_x, self.len_y), dtype=str)

        for index, char in self.canevas.items():
            canevas[tuple([sum(x) for x in zip(self.offset, index)])] = char
        
        for s in self.tp.values():
            for pos in s:
                canevas[pos] = 'X'
                 
        print(canevas)
    
    def parse_map(self, path):
        with open(path, "r") as f:
            i = 0
            for line in f:
                if line != "":
                    for j, char in enumerate(line):
                        if char == "\n":
                            continue

                        self.canevas[(i, j)] = char
                        
                        if char == ".":
                            for level in range(self.nb_levels):
                                self.G.add_node((i, j, level))
                                
                            if self.canevas.get((i-1, j), "#") == ".":
                                for level in range(self.nb_levels):
                                    self.G.add_edge((i-1, j, level), (i, j, level))
                            
                            if self.canevas.get((i, j-1), "#") == ".":
                                for level in range(self.nb_levels):
                                    self.G.add_edge((i, j-1, level), (i, j, level))
                    i += 1
        
                    
        min_x = min(self.canevas, key=lambda k: k[0])[0]
        max_x = max(self.canevas, key=lambda k: k[0])[0]
        min_y = min(self.canevas, key=lambda k: k[1])[1]
        max_y = max(self.canevas, key=lambda k: k[1])[1]
        
        
        self.offset = (abs(min_x), abs(min_y))
        
        self.len_x = max_x - min_x + 1
        self.len_y = max_y - min_y + 1
        
                    
    def add_tp(self):
        for pos, char in self.canevas.items():
            i, j = pos
            if char.isalpha():
                if self.canevas.get((i-1, j), '?').isalpha() and self.canevas.get((i-2, j), '?') == '.':
                    self.tp[(self.canevas[i-1, j], char)].append((i-2, j))
                if self.canevas.get((i+1, j), '?').isalpha() and self.canevas.get((i+2, j), '?') == '.':
                    self.tp[(char, self.canevas[i+1, j])].append((i+2, j))
                if self.canevas.get((i, j-1), '?').isalpha() and self.canevas.get((i, j-2), '?') == '.':
                    self.tp[(self.canevas[i, j-1], char)].append((i, j-2))
                if self.canevas.get((i, j+1), '?').isalpha() and self.canevas.get((i, j+2), '?') == '.':
                    self.tp[(char, self.canevas[i, j+1])].append((i, j+2))
            
        for tps, positions in self.tp.items():
            if len(positions) > 1:
                if positions[0][0] in [2, self.len_x-3] or positions[0][1] in [2, self.len_y-3]:
                    outer, inner = positions
                else:
                    inner, outer = positions
                
                for level in range(self.nb_levels - 1):
                    self.G.add_edge((*inner, level), (*outer, level + 1))
                    self.G.add_edge((*outer, level + 1), (*inner, level))
            else:
                self.start_end.extend(list(positions))
                
        
    def part_2(self, path):
        self.parse_map(path)
        self._map()
        self.add_tp()
        print(self.tp)
        start, end = self.start_end
        return nx.shortest_path_length(self.G,(*start, 0), (*end, 0))

In [ ]:
tpm2 = TPMaster2()
tpm2.part_2("/home/bchatillon/Documents/Advent-of-Code/2019/20_data.txt")

In [ ]:
b